### Proyecto Final - Ingenierìa de Datos

Para el proyecto se usarà informaciòn de la tienda ficticia Superstore, una cadena estadounidense que se encuentra en varios estados que ofrece una gran cantidad de productos. la tienda quiere empezar a utilizar su informaciòn para la toma de decisiones, y para esto requiere que se estructure de forma tal que permita hacerlo. Como cualquier negocio nuevo en esto, no saben exactamente por donde empezar, por lo que han contratado nuestro servicio para ayudarlos en entender lo que se tiene y como se puede utilizar.

Luego de algunas reuniones se logra entender la data actual del negocio, los datos actualmente se encuentran en archivos csv separados por varias categorìas, como productos, clientes, ubicaciones, entre otros. Adicional cuentan con una tabla que tiene los registros de todas las compras. Estos archivos de dimensiones son actualizados eventualmente, pero sufren pocos cambios. El que tiene nuevos registros con màs constancia es el de clienetes. El documento con la informaciòn de las transacciones sì tiene nueva informaciòn todos los dìas.


Evaluando varias alternativas, se decide que se utilizaràn los servicios que se encuentran disponibles en Amazon Web Services, ya que permite almacenar archivos planos, creaciòn de data bases y data warehouses y podemos conectar todo usando scripts de python.  

El proyecto con la tienda Superstore, se basarà en subir todos los archivos a un bucket de S3, en diferentes carpetas segùn la naturaleza de la informaciòn, para posteriormente guardarlos en una base de datos de RDS, ya que estos estàn en primer o segunda forma normal. Luego, se haràn las transformaciones necesarias para unificar las dimensiones y se cargarà todo a un data warehouse, desde donde podremos conectar nuestra herramienta de inteligencia de negocio para empezar a responder preguntas sobre el comportamiento de nuestra informaciòn.

El negocio principalmente quiere saber lo siguiente:

1. En dònde en el paìs se genera la mayor ganancia, para enfocarse en no descuidar esas àreas
2. Què segmento de cliente es el que menos compra, para pensar en una campaña enfocada en ganar ese mercado
3. En què meses del año la demanda de productos es mayor, para anticipar el tema de inventarios
4. Los descuentos que se dan segùn el tipo de envio realizado, para no restar margen a los envìos que màs cuestan a la empresa
5. Evaluar si el producto que màs se vende, es el que màs ganancias genera

##### Dentro de AWS

Ya en AWS, se puede observar que la informaciòn quedò guardada de la siguiente forma:

Archivos en bucket de S3, segùn el tipo de archivo, se almacena en una carpeta diferente, donde iràn depositando los nuevos que se generen

<img src="imagen_1.png">

Bases de datos en RDS, se crea la base de datos inicial, donde irà la informaciòn de los archivos en una tabla diferente cada uno, y luego un DW con mysql, donde iràn las tablas que correspondan unidas, para anàlisis en herramienta de BI.

<img src="imagen_2.png">

##### Dentro del Gestor para bases de datos

Se utilizò DBeaver como gestor de base de datos, y la base de datos y el data warehouse quedaron de la siguiente forma.

Base de datos

<img src="imagen_3.png">

Data Warehouse

<img src="imagen_4.png">

##### Respondiendo las preguntas del negocio

Para responder las preguntas de negocio, leeremos la informaciòn en el DW usando python y tambièn nos ayudaremos generando visuales a travès de Tableau, leyendo esta misma base de datos.

Se accede al DW en Tableau

<img src="imagen_10.png">

In [1]:
# carga de librerias

import pandas as pd
import numpy as np
import boto3
import psycopg2
import configparser
import io
from sqlalchemy import create_engine
import pymysql

In [2]:
# cargar archivos de configuraciones

config = configparser.ConfigParser()
config.read("credenciales_proyecto.cfg")

['credenciales_proyecto.cfg']

In [3]:
# crear driver

mysql_driver = f"""mysql+pymysql://{config.get('RDS_MYSQL', 'DB_USER')}:{config.get('RDS_MYSQL', 'DB_PASSWORD')}@{config.get('RDS_MYSQL', 'DB_HOST')}:{config.get('RDS_MYSQL', 'DB_PORT')}/{config.get('RDS_MYSQL', 'DB_NAME')}"""

In [4]:
# importar las tablas del data warehouse y guardarlas en dataframes

sql_query_retail_sales_fact = "SELECT * FROM retail_sales_fact;"
df_retail_sales_fact = pd.read_sql(sql_query_retail_sales_fact, mysql_driver)

sql_query_product_dimension = "SELECT * FROM product_dimension;"
df_product_dimension = pd.read_sql(sql_query_product_dimension, mysql_driver)

sql_query_customer_dimension = "SELECT * FROM customer_dimension;"
df_customer_dimension = pd.read_sql(sql_query_customer_dimension, mysql_driver)

sql_query_ship_mode = "SELECT * FROM ship_mode;"
df_ship_mode = pd.read_sql(sql_query_ship_mode, mysql_driver)

sql_query_location_dimension = "SELECT * FROM location_dimension;"
df_location_dimension = pd.read_sql(sql_query_location_dimension, mysql_driver)

sql_query_date_dimension = "SELECT * FROM date_dimension;"
df_date_dimension = pd.read_sql(sql_query_date_dimension, mysql_driver)


##### Pregunta 1

En donde en el paìs se genera la mayor ganancia

In [13]:
df_pregunta_1 = (
    df_retail_sales_fact.merge(df_location_dimension, on="location_key", how="inner")[
        ["city", "profit"]
    ]
    .groupby("city")
    .sum("profit")
    .sort_values("profit", ascending=False)
)

In [14]:
df_pregunta_1

,profit
city,
New York City,62036.86
Los Angeles,30440.67
Seattle,29156.08
San Francisco,17507.30
Detroit,13181.77
...,...
Chicago,-6654.63
Lancaster,-7239.08
San Antonio,-7299.07


Se carga la informaciòn en una herramienta de visualizaciòn para una mejor exploraciòn, vemos que Nueva York, los Angeles y Seattle son las ciudades que màs ganancia nos dan

<img src="imagen_5.png">

##### Pregunta 2

Què segmento de cliente es el que menos compra

In [16]:
df_pregunta_2 = (
    df_retail_sales_fact.merge(df_customer_dimension, on="customer_key", how="inner")[
        ["segment", "sales"]
    ]
    .groupby("segment")
    .sum("sales")
    .sort_values("sales", ascending=True)
)

In [17]:
df_pregunta_2

,sales
segment,
Home Office,429653.19
Corporate,706146.36
Consumer,1161401.23


Vemos que el segmento de Home Office son los clientes que menos nos compran

<img src="imagen_6_1.png">

##### Pregunta 3

En què meses del año la demanda es mayor de productos

In [23]:
df_pregunta_3 = (
    df_retail_sales_fact.merge(df_date_dimension, on="date_key", how="inner")[
        ["month_", "month_name", "quantity"]
    ]
    .groupby(["month_", "month_name"])
    .sum("quantity")
    .sort_values("quantity", ascending=False)
)

In [24]:
df_pregunta_3

,,quantity
month_,month_name,
11,November,5775
12,December,5419
9,September,5062
10,October,3104
5,May,2791
8,August,2784
7,July,2705
6,June,2680
3,March,2564


Con ayuda de la gràfica identificamos que noviembre, diciembre y septiembre son los meses que màs unidades de nuestros productos se compran

<img src="imagen_7.png">

##### Pregunta 4

Descuentos que se dan segùn tipo de envìò realizado

In [5]:
df_pregunta_4 = (
    df_retail_sales_fact.merge(df_ship_mode, on="ship_mode_key", how="inner")[
        ["ship_mode", "discount"]
    ]
    .groupby("ship_mode")
    .sum("discount")
    .sort_values("discount", ascending=False)
)

In [6]:
df_pregunta_4

,discount
ship_mode,
Standard Class,955.02
Second Class,270.15
First Class,253.17
Same Day,82.75


Se està dando la mayor cantidad de descuentos en la clase de envìo estàndar y la menor en los envìos del mismo dìa

<img src="imagen_8.png">

##### Pregunta 5

Evaluar si el producto que màs se vende es el que màs ganancia deja

In [9]:
df_pregunta_5 = (
    df_retail_sales_fact.merge(df_product_dimension, on="product_key", how="inner")[
        ["sub_category", "sales", "profit"]
    ]
    .groupby("sub_category")
    .sum(["profit", "sales"])
    .sort_values("sales", ascending=False)
)

In [15]:
df_pregunta_5[df_pregunta_5["sales"]==df_pregunta_5["sales"].max()]

,sales,profit
sub_category,,
Phones,330007.1,44515.99


In [16]:
df_pregunta_5[df_pregunta_5["profit"]==df_pregunta_5["profit"].max()]

,sales,profit
sub_category,,
Copiers,149528.01,55617.87


In [10]:
df_pregunta_5

,sales,profit
sub_category,,
Phones,330007.10,44515.99
Chairs,328449.07,26590.11
Storage,223843.59,21278.83
Tables,206965.57,-17725.59
Binders,203412.76,30221.41
Machines,189238.66,3384.73
Accessories,167380.31,41936.72
Copiers,149528.01,55617.87
Bookcases,114879.97,-3472.58


Encontramos que el producto que màs se vende, que es telèfonos, no es el que deja una mayor ganancia. Un producto con la mitad de las ventas, copiadoras, es el que mayor ganancia da, luego junto con la tabla y gràfica podemos ver otros comportamientos curiosos

<img src="imagen_9.png">

*Todos los anàlisis se hacen sobre el total de data disponible de todos los años